In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from IPython.display import Markdown, display

In [4]:
# !pip install pymysql

In [7]:
# !pip install llama_index

In [8]:
# pip install --force-reinstall 'sqlalchemy<2.0.0'

In [9]:
import pymysql

## Connect to Database

In [10]:
db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "classicmodels"

In [11]:
from sqlalchemy import create_engine, text

# Construct the connection string
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}?charset=utf8"

# Create an engine instance
engine = create_engine(connection_string)

# Test the connection using raw SQL
with engine.connect() as connection:
    result = connection.execute(text("show tables"))
    for row in result:
        print(row)

('customers',)
('employees',)
('offices',)
('orderdetails',)
('orders',)
('payments',)
('productlines',)
('products',)


## Load database to llamaIndex SQLDatabase

In [ ]:
# !pip uninstall llama_index
# !pip install llama_index
# !pip install llama_index==0.5.0


In [9]:
from llama_index.core import SQLDatabase

In [10]:
tables = ['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'productlines', 'products']
sql_database = SQLDatabase(engine, include_tables=tables, sample_rows_in_table_info=2)
sql_database

In [11]:
sql_database._all_tables

{'customers',
 'employees',
 'offices',
 'orderdetails',
 'orders',
 'payments',
 'productlines',
 'products'}

In [12]:
sql_database.get_single_table_info

<bound method SQLDatabase.get_single_table_info of <llama_index.core.utilities.sql_wrapper.SQLDatabase object at 0x0000025D59E22CD0>>

## Connect to OpenAI account

In [13]:
import os
import openai

In [ ]:
openai.api_key = ""

In [ ]:
# !pip install setuptools cython
# !pip install aimrocks
# !pip install llama-index-callbacks-aim
# !pip install --upgrade pip setuptools wheel
# !pip install llama-index-callbacks-aim --no-deps
# !pip install aimrocks
# !pip install other-required-packages
# !pip install --upgrade llama-index
# !pip install --upgrade llama-index

# !pip install llama-index==0.6.0


In [16]:

# !pip show llama_index

# !pip install llama_index==0.5.0
# !pip install langchain_community

# !pip install --upgrade llama_index
# !pip install --upgrade llama_index
# !pip install llama_index==0.5.16
# !pip install llama_index[langchain]
# !pip install llama_index==0.5.0
# !pip install --upgrade langchain
# !pip install llama-index llama-index-llms-ollama
# !pip install --upgrade langchain


## Connect to LLM and initialize the service context

In [17]:
import tiktoken
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

callback_manager = CallbackManager([token_counter])

In [18]:

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900




## Create SQL table node mapping

In [19]:
#creating SQL table node mapping
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex, SQLTableNodeMapping, SQLTableSchema
import pandas as pd

# list all the tables from database and crate table schema for prompt to LLM
tables = list(sql_database._all_tables)
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = []
for table in tables:
    table_schema_objs.append((SQLTableSchema(table_name = table)))

In [20]:
print(table_schema_objs)

[SQLTableSchema(table_name='customers', context_str=None), SQLTableSchema(table_name='payments', context_str=None), SQLTableSchema(table_name='productlines', context_str=None), SQLTableSchema(table_name='orderdetails', context_str=None), SQLTableSchema(table_name='orders', context_str=None), SQLTableSchema(table_name='products', context_str=None), SQLTableSchema(table_name='employees', context_str=None), SQLTableSchema(table_name='offices', context_str=None)]


## Initialize SQL Table Retrieval Engine

In [21]:
from llama_index.core.indices.struct_store.sql_query import SQLTableRetrieverQueryEngine

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [22]:
# Create the query engine
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=3), service_context=Settings
)

In [23]:
response = query_engine.query("How many tables inclassimodels  database?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'customers' has columns: customerNumber (INTEGER), customerName (VARCHAR(50)), contactLastName (VARCHAR(50)), contactFirstName (VARCHAR(50)), phone (VARCHAR(50)), addressLine1 (VARCHAR(50)), addressLine2 (VARCHAR(50)), city (VARCHAR(50)), state (VARCHAR(50)), postalCode (VARCHAR(15)), country (VARCHAR(50)), salesRepEmployeeNumber (INTEGER), creditLimit (DECIMAL(10, 2)),  and foreign keys: ['salesRepEmployeeNumber'] -> employees.['employeeNumber'].

Table 'orders' has columns: orderNumber (INTEGER), orderDate (DATE), requiredDate (DATE), shippedDate (DATE), status (VARCHAR(15)), comments (TEXT), customerNumber (INTEGER),  and foreign keys: ['customerNumber'] -> customers.['customerNumber'].

Table 'employees' has columns: employeeNumber (INTEGER), las

In [24]:
print(response)

There are 0 tables in the inclassmodels database.


In [ ]:
import openai
import pandas as pd
from sqlalchemy import create_engine, text
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core.indices.struct_store.sql_query import SQLTableRetrieverQueryEngine
from llama_index.core.objects import ObjectIndex, SQLTableNodeMapping, SQLTableSchema
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter


# Set your OpenAI API key
openai.api_key = ""  # Replace with your actual OpenAI API key


# Database connection details
db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "classicmodels"

# Create SQLAlchemy engine
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}?charset=utf8"
engine = create_engine(connection_string)

# Create SQLDatabase object
from llama_index.core import SQLDatabase
sql_database = SQLDatabase(engine)

# List all tables in the database
tables = sql_database._all_tables

# Create SQL table schema descriptions
table_schema_objs = []
for table in tables:
    table_schema_objs.append(SQLTableSchema(table_name=table))

# Initialize ObjectIndex and SQLTableNodeMapping
table_node_mapping = SQLTableNodeMapping(sql_database)
obj_index = ObjectIndex.from_objects(table_schema_objs, table_node_mapping, VectorStoreIndex)

# Define the SQL query generation function
def generate_sql_query(query_str):
    """
    Given a natural language question, generate the correct SQL query for the database.
    This uses LlamaIndex's query engine to generate the SQL query from the input question.
    """
    # Define the prompt for the model with a clear instruction to only generate SQL query
    prompt = f"""
    You are an agent designed to interact with a SQL database. 
    Given the following question, generate only the correct SQL query to retrieve the requested information.
    
    Question: {query_str}
    SQL Query (only the query, no explanations or additional text):
    """

    # Initialize the OpenAI model for query generation
    Settings.llm = OpenAI(model="gpt-3.5-turbo")  # Set model to use

    # Initialize SQL Table Retrieval Engine
    query_engine = SQLTableRetrieverQueryEngine(
        sql_database, obj_index.as_retriever(similarity_top_k=3), service_context=Settings
    )

    # Query the engine to generate a SQL query based on the prompt
    sql_query_response = query_engine.query(prompt)

    # Check if the response has a 'text' attribute and extract the query
    if hasattr(sql_query_response, 'text'):
        sql_query = sql_query_response.text.strip()  # Extract the SQL query text and remove extra spaces
    else:
        # If the response does not have a 'text' attribute, fall back to using str() for extraction
        sql_query = str(sql_query_response).strip()

    # Return the clean SQL query
    return sql_query

# Example natural language query (question)
query_str = "How many employees do we have?"

# Generate the SQL query using the question
generated_sql_query = generate_sql_query(query_str)

# Print the generated SQL query
print(f"Generated SQL Query: {generated_sql_query}")

# Execute the generated SQL query using SQLAlchemy
try:
    with engine.connect() as connection:
        result = connection.execute(text(generated_sql_query))  # Execute SQL query
        df = pd.DataFrame(result.fetchall(), columns=result.keys())  # Convert to DataFrame
        print(df)  # Display the results in a DataFrame
except Exception as e:
    print(f"Error executing SQL: {e}")


Generated SQL Query: There are 23 employees in the database.
Error executing SQL: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'There are 23 employees in the database.' at line 1")
[SQL: There are 23 employees in the database.]
(Background on this error at: https://sqlalche.me/e/20/f405)
